In [1]:
import os
os.chdir('E:\ML_Labs')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
#from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Reshape, Dense, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
#from keras.callbacks import Callback

In [3]:
class PC:
    @staticmethod
    def build(time_steps, num_features, reg=0.0005):
        # initialize the input shape to be "channels last" and the
        # channels dimension itself
        # define the model input and first CONV module
        inputs = Input(shape=(time_steps, num_features))
        lstm=Bidirectional(LSTM(48,return_sequences=True))(inputs)
        lstm=LSTM(48)(lstm)
        lstm=Activation('tanh')(lstm)
        lstm=Dense(24)(lstm)
        # create the model
        model = Model(inputs, lstm, name="PC")
        
        # return the constructed network architecture
        return model

In [4]:
model1 =  PC.build(time_steps=24, num_features=21, reg=0.005)

In [5]:
model1.summary()

Model: "PC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 24, 96)         │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 48)             │        27,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         1,176 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,896 (218.34 KB)

 Trainable params: 55,896 (218.34 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
checkpoints = r'E:\ML_Labs\venv\lab2\\E3-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:\ML_Labs\venv\lab2'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\h2istory.json"])

In [7]:
model = None
start_epoch = 0
time_steps=24
num_features=21

In [8]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [9]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [10]:
import pickle
import os
path_dataset =r'E:\ML_Labs\venv\Lab_2\orignal_AEP_hourly'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(r'E:\ML_Labs\venv\Lab_2\AEP_train.csv')
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(r'E:\ML_Labs\venv\Lab_2\AEP_validation.csv')
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(r'E:\ML_Labs\venv\Lab_2\AEP_test.csv')
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(r'E:\ML_Labs\venv\Lab_2\AEP_Scaler.pkl', 'rb'))

train_set.shape, validation_set.shape, test_set.shape

e:\ML_Labs\venv\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [11]:
time_steps=24
num_features=21

In [12]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=24)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=24)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=24)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.1796350479125977 sec


In [13]:
epochs = 2
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/2
2651/2652 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0637 - mae: 0.0637 - mape: 1847.4189
Epoch 1: val_loss improved from inf to 0.03943, saving model to E:\ML_Labs\venv\lab2\\E3-cp-0001-loss0.04.h5


2652/2652 ━━━━━━━━━━━━━━━━━━━━ 90s 31ms/step - loss: 0.0637 - mae: 0.0637 - mape: 1847.6700 - val_loss: 0.0394 - val_mae: 0.0394 - val_mape: 16.7484
Epoch 2/2
2651/2652 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0373 - mae: 0.0373 - mape: 1862.9000
Epoch 2: val_loss improved from 0.03943 to 0.03718, saving model to E:\ML_Labs\venv\lab2\\E3-cp-0002-loss0.04.h5


2652/2652 ━━━━━━━━━━━━━━━━━━━━ 103s 39ms/step - loss: 0.0373 - mae: 0.0373 - mape: 1863.1180 - val_loss: 0.0372 - val_mae: 0.0372 - val_mape: 16.2597


In [14]:
model = load_model(r'E:\ML_Labs\venv\lab2\\E3-cp-0002-loss0.04.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

378/378 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step
Mean Absolute Error (MAE): 595.4
Median Absolute Error (MedAE): 457.23
Mean Squared Error (MSE): 617513.87
Root Mean Squared Error (RMSE): 785.82
Mean Absolute Percentage Error (MAPE): 4.02 %
Median Absolute Percentage Error (MDAPE): 3.19 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)


In [15]:
checkpoints =r'E:\ML_Labs\venv\lab2\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ML_Labs\venv\lab2\\E3-cp-0002-loss0.04.h5'
start_epoch= 5

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print("[INFO] loading model from disk...")
    model = load_model(r'E:\ML_Labs\venv\E1-cp-0002-loss0.02.h5', compile=False)

    # Manually compile since we used compile=False
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

    # Print and update learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))
    model.optimizer.learning_rate = 1e-4

    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.learning_rate)))


[INFO] loading model from disk...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [17]:
epochs = 2
verbose = 1 #0
batch_size = 128
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/2
660/663 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0971 - mae: 0.0971 - mape: 6645.1602
Epoch 1: val_loss did not improve from 0.03718
663/663 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0971 - mae: 0.0971 - mape: 6636.8857 - val_loss: 0.0911 - val_mae: 0.0911 - val_mape: 54.1220
Epoch 2/2
649/663 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0903 - mae: 0.0903 - mape: 5709.0376
Epoch 2: val_loss did not improve from 0.03718
663/663 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0903 - mae: 0.0903 - mape: 5702.7432 - val_loss: 0.0907 - val_mae: 0.0907 - val_mape: 54.2605


In [18]:

model = load_model(r'E:\ML_Labs\venv\lab2\\E3-cp-0002-loss0.04.h5', compile=False)

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

378/378 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step
Mean Absolute Error (MAE): 595.4
Median Absolute Error (MedAE): 457.23
Mean Squared Error (MSE): 617513.87
Root Mean Squared Error (RMSE): 785.82
Mean Absolute Percentage Error (MAPE): 4.02 %
Median Absolute Percentage Error (MDAPE): 3.19 %


y_test_unscaled.shape=  (12082, 24)
y_pred.shape=  (12082, 24)
